In [0]:
# Task 1
dbutils.fs.cp("file:/Workspace/Shared/sales1_data.csv", "dbfs:/FileStore/sales1_data.csv")

dbutils.fs.cp("file:/Workspace/Shared/customer_data.json", "dbfs:/FileStore/customer_data.json")

dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")


# Load it into a DataFrame
df_sales = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/sales1_data.csv")

df_customers = spark.read.option("multiline", "true").json("dbfs:/FileStore/customer_data.json")

# Write the Dataframe as delta table
df_sales.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("dbfs:/FileStore/sales_delta")


df_customers.write.format("delta").mode("overwrite").save("dbfs:/FileStore/customers_delta")

# Convert Parquet file into delta table
df_sales.write.format("parquet").mode("overwrite").save("dbfs:/FileStore/sales_parquet")

df_parquet = spark.read.format("parquet").load("dbfs:/FileStore/sales_parquet")

df_parquet.write.format("delta").mode("overwrite").save("dbfs:/FileStore/sales_delta_parquet")




In [0]:
# Task 2
# Load new sales into the daata frame
df_new_sales = spark.read.format("csv").option("header","true").load("dbfs:/FileStore/new_sales_data.csv")

# Write into delta table
df_new_sales.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("dbfs:/FileStore/new_sales_delta")


sales_df = spark.read.format("delta").load("dbfs:/FileStore/sales_delta")
new_sales_df = spark.read.format("delta").load("dbfs:/FileStore/new_sales_delta")

sales_df.createOrReplaceTempView("delta_sales")
new_sales_df.createOrReplaceTempView("delta_new_sales")

spark.sql("""
        MERGE INTO delta_sales as target
        USING delta_new_sales as source
        ON target.OrderID = source.OrderID
        WHEN MATCHED THEN
        UPDATE SET
                target.OrderDate = source.OrderDate,
                target.CustomerID = source.CustomerID,
                target.Product = source.Product,
                target.Quantity = source.Quantity,
                target.Price = source.Price
        WHEN NOT MATCHED THEN
        INSERT (OrderID, OrderDate, CustomerID, Product, Quantity, Price)
        VALUES (source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price);  
""")

spark.sql("SELECT * FROM delta_sales").show()

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
+-------+----------+----------+--------+--------+-----+



In [0]:
# Task 3
# Register the Delta table
spark.sql("CREATE TABLE IF NOT EXISTS sales_delta_table USING DELTA LOCATION 'dbfs:/FileStore/sales_delta'")

# Optimize
spark.sql("OPTIMIZE sales_delta_table ZORDER BY (CustomerID)")


+--------------------+--------------------+
|                path|             metrics|
+--------------------+--------------------+
|dbfs:/FileStore/s...|{0, 0, {NULL, NUL...|
+--------------------+--------------------+



In [0]:
# Task 4
# Advanced Features

# DESCRIBE HISTORY
spark.sql("DESCRIBE HISTORY sales_delta_table").show()

# VACUUM
spark.sql("VACUUM sales_delta_table RETAIN 168 HOURS")

+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      9|2024-09-13 07:16:19|6822499424790006|azuser2121_mml.lo...| OPTIMIZE|{predicate -> [],...|NULL|{3562288518827637}|0911-073432-r1idfcx3|          8|SnapshotIsolation|        false|{numRemovedFiles ...|        NULL|Databricks-Runtim...|
|      8|2024-09-13 07:16:16

DataFrame[path: string]

In [0]:
# Task 5

# Delta lake time travel
spark.sql("SELECT * FROM sales_delta_table VERSION AS OF 0").show()


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
+-------+----------+----------+--------+--------+-----+



In [0]:
# Building a reliable Data Lake with Delta Lake
from pyspark.sql.types import StructType, StructField, StringType, DateType
# Schema enforcement
customer_schema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("CustomerName", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("SignupDate", StringType(), True)
])

customer_df = spark.read.json("dbfs:/FileStore/customer_data.json",schema=customer_schema)

customer_df.write.format("delta").mode("overwrite").save("dbfs:/FileStore/customers_delta")

new_customer_data = [
    ("C006", "Alex Wilson", "East", "2024-02-01"),
    ("C002", "Jane Smith", "West", "2023-02-15") 
]

new_customer_df = spark.createDataFrame(new_customer_data, schema=customer_schema)
new_customer_df.createOrReplaceTempView("new_customer_data")

customer_df = spark.read.format("delta").load("dbfs:/FileStore/customers_delta")
customer_df.createOrReplaceTempView("customer_delta_table")

print("Merging new customer data into the Delta Lake")
merge_query = f"""
MERGE INTO customer_delta_table AS target
USING new_customer_data AS source
ON target.CustomerID = source.CustomerID
WHEN MATCHED THEN
  UPDATE SET
    CustomerName = source.CustomerName,
    Region = source.Region,
    SignupDate = source.SignupDate
WHEN NOT MATCHED THEN
  INSERT (CustomerID, CustomerName, Region, SignupDate)
  VALUES (source.CustomerID, source.CustomerName, source.Region, source.SignupDate)
"""

spark.sql(merge_query)

# Optimize
print("Optimizing")
spark.sql("CREATE TABLE IF NOT EXISTS customer_delta_table1 USING DELTA AS SELECT * FROM customer_delta_table")

spark.sql("OPTIMIZE customer_delta_table1 ZORDER BY (Region)")

# Vacuum
print("Vacuuming")
spark.sql("VACUUM customer_delta_table1 RETAIN 168 HOURS")

# Display the delta lake
spark.sql("SELECT * FROM customer_delta_table1").show()

Merging new customer data into the Delta Lake
Optimizing
Vacuuming
+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
|      C006|  Alex Wilson|  East|2024-02-01|
|      C002|   Jane Smith|  West|2023-02-15|
+----------+-------------+------+----------+

